In [7]:
from bs4 import BeautifulSoup
import requests
import time, os

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import pandas as pd

chromedriver = '../chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver

In [9]:
links = []

response = requests.get('https://www.nytimes.com/interactive/2021/us/covid-cases.html')
page = response.text
soup = BeautifulSoup(page, 'lxml')
state_links = soup.find('ul', class_ = 'secondary svelte-173co1c').find_all('li')
for state in state_links:
    link_ = state.find('a')['href']
    links.append('https://www.nytimes.com' + link_)

DC = links[-4]
links.remove(DC)
links

['https://www.nytimes.com/interactive/2021/us/alabama-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/alaska-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/arizona-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/arkansas-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/california-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/colorado-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/connecticut-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/delaware-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/florida-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/georgia-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/hawaii-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/idaho-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/us/illinois-covid-cases.html',
 'https://www.nytimes.com/interactive/2021/u

In [10]:
response = requests.get('https://www.nytimes.com/interactive/2021/us/washington-covid-cases.html')
response.status_code

200

In [11]:
nytimes_loginpage = 'https://myaccount.nytimes.com/auth/login?response_type=cookie&client_id=vi&redirect_uri=https%3A%2F%2Fwww.nytimes.com%2Fsubscription%2Fmultiproduct%2Flp8KQUS.html%3FcampaignId%3D7JFJX%26EXIT_URI%3Dhttps%253A%252F%252Fwww.nytimes.com%252F&asset=masthead'

In [16]:
chromedriver = 'chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver
options = Options()
#options.add_argument('--headless')
driver = webdriver.Chrome(chromedriver, chrome_options=options)
driver.maximize_window()
    
driver.get(nytimes_loginpage)

alltime_xpath = '//section[@id=\'county-trends\']/div[@class=\'multi-buttons\']/button[2]'
showall_xpath = '//section[@id=\'county-trends\']/button[@class=\'showall\']'

driver.find_element_by_id('username').send_keys('vaqaasaslam@gmail.com') # user/pass used to log-in, but removed after csv saved
driver.find_element_by_id('password').send_keys('Syedv786', Keys.ENTER) # for privacy.

time.sleep(5)

driver.get(links[0])

alltime_elem = driver.find_element_by_xpath(alltime_xpath)
showall_elem = driver.find_element_by_xpath(showall_xpath)

driver.execute_script("arguments[0].click();", alltime_elem) 
driver.execute_script("arguments[0].click();", showall_elem)

<ipython-input-16-13a499d5a9f1>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver, chrome_options=options)


In [13]:
def make_alltime_list(url, driver):
    
    driver.get(url)

    xpath1 = '//section[@id=\'county-trends\']/div[@class=\'multi-buttons\']/button[2]'
    xpath2 = '//section[@id=\'county-trends\']/button[@class=\'showall\']'
    elem1 = driver.find_element_by_xpath(xpath1)
    try:
        elem2 = driver.find_element_by_xpath(xpath2)
    except:
        print('No Show All button')
    
    driver.execute_script("arguments[0].click();", elem1) 
    try:
        driver.execute_script("arguments[0].click();", elem2)
    except:
        print('No Show All button to click')
    
    time.sleep(5)
    
    page = driver.page_source    
    soup = BeautifulSoup(page, 'lxml')
    
    table = soup.find(class_ = 'g-table super-table withchildren')
    
    table_body = table.find('tbody', class_ = 'children')
    table_rows = table_body.find_all('tr')
    
    row_data = []
    for row in table_rows:
        data = []
        tds = row.find_all('td')
        for td in tds:
            if td['class'][0] == 'chart':
                pass
            else:
                data.append(td.text)
        row_data.append(data)
        
    
    return row_data

def make_recent_trend_list(url, driver):
    
    driver.get(url)
    
    xpath = '//section[@id=\'county-trends\']/button[@class=\'showall\']'
    try:
        elem = driver.find_element_by_xpath(xpath)
    except:
        print('No Show All button')
    try:
        driver.execute_script("arguments[0].click();", elem)
    except:
        print('No Show All button to click')
    
    time.sleep(5)
    
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    
    table = soup.find(class_ = 'g-table super-table withchildren')
    
    table_body = table.find('tbody', class_ = 'children')
    table_rows = table_body.find_all('tr')
    
    row_data = []
    for row in table_rows:
        data = []
        tds = row.find_all('td')
        for td in tds:
            if td['class'][0] == 'chart':
                pass
            else:
                data.append(td.text)
        data.append(row.find('span', class_ = 'pct-change').text)
        
        row_data.append(data)
        
    return row_data

In [14]:
#################################################
#### This cell runs for about 30-40 minutes ####
###############################################

chromedriver = 'chromedriver'
os.environ['webdriver.chrome.driver'] = chromedriver
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(chromedriver, chrome_options=options)
driver.maximize_window()
    
driver.get(nytimes_loginpage)

alltime_xpath = '//section[@id=\'county-trends\']/div[@class=\'multi-buttons\']/button[2]'
showall_xpath = '//section[@id=\'county-trends\']/button[@class=\'showall\']'

driver.find_element_by_id('username').send_keys('vaqaasaslam@gmail.com')
driver.find_element_by_id('password').send_keys('Syedv786', Keys.ENTER)

time.sleep(5)
all_data = []
recent_data = []
for link in links:
    data = make_alltime_list(link, driver)
    all_data.extend(data)
    
for link in links:
    data = make_recent_trend_list(link, driver)
    recent_data.extend(data)

<ipython-input-14-140940dfc90e>:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver, chrome_options=options)


No Show All button
No Show All button to click
No Show All button
No Show All button to click
No Show All button
No Show All button to click
No Show All button
No Show All button to click
No Show All button
No Show All button to click
No Show All button
No Show All button to click
No Show All button
No Show All button to click
No Show All button
No Show All button to click
No Show All button
No Show All button to click
No Show All button
No Show All button to click


In [17]:
recent_df = pd.DataFrame(recent_data, columns = ['County','Daily Case Avg',
                                     'Daily Case 14-Day change %', 'Hospitalized Daily Avg', 'Hosp. Daily per 100k',
                                     'Hosp. Daily change %', 'Deaths Daily Avg.', 'Deaths Daily per 100k',
                                     'Fully Vaccinated'])

In [18]:
all_df = pd.DataFrame(all_data, columns = ['County','Total Cases', 'Total Cases per 100k',
                                   'Total Deaths', 'Total Deaths per 100k','Fully Vaccinated'])

In [24]:
recent_df = recent_df.sort_values(by=['County'])
all_df = all_df.sort_values(by=['County'])
recent_df.head()

,County,Daily Case Avg,Daily Case 14-Day change %,Hospitalized Daily Avg,Hosp. Daily per 100k,Hosp. Daily change %,Deaths Daily Avg.,Deaths Daily per 100k,Fully Vaccinated
2429,Abbeville ›,1,5,12,+11%,0,—,26%,–40%
1136,Acadia ›,7,11,11,–3%,0,—,24%,+3%
2992,Accomack ›,2,5,6,–31%,0,—,—,–54%
848,Adair ›,<1,6,13,+99%,0,—,34%,–73%
1588,Adair ›,0,—,31,+4%,0,—,30%,–100%


In [25]:
all_df.head()

,County,Total Cases,Total Cases per 100k,Total Deaths,Total Deaths per 100k,Fully Vaccinated,Daily Case Avg,Daily Case 14-Day change %,Hospitalized Daily Avg,Hosp. Daily per 100k,Hosp. Daily change %,Deaths Daily Avg.,Deaths Daily per 100k
2423,Abbeville ›,"2,576","10,503",41,167,26%,3,9,8,–55%,0.1,0.47,32%
1138,Acadia ›,"6,545","10,549",192,309,24%,15,10,8,+10%,0,—,20%
2939,Accomack ›,"2,830","8,757",41,127,—,13,5,7,–34%,0.6,0.23,—
2131,Adair ›,"3,181","14,333",41,185,18%,6,7,7,–19%,0.1,0.18,27%
822,Adair ›,865,"12,095",32,447,34%,1,9,14,+99%,0.1,1.34,39%


In [21]:
columns = ['Daily Case Avg','Daily Case 14-Day change %', 'Hospitalized Daily Avg',
           'Hosp. Daily per 100k', 'Hosp. Daily change %','Deaths Daily Avg.', 'Deaths Daily per 100k']

for column in columns:
    all_df[column] = recent_df[column]

In [26]:
all_df.head()

,County,Total Cases,Total Cases per 100k,Total Deaths,Total Deaths per 100k,Fully Vaccinated,Daily Case Avg,Daily Case 14-Day change %,Hospitalized Daily Avg,Hosp. Daily per 100k,Hosp. Daily change %,Deaths Daily Avg.,Deaths Daily per 100k
2423,Abbeville ›,"2,576","10,503",41,167,26%,3,9,8,–55%,0.1,0.47,32%
1138,Acadia ›,"6,545","10,549",192,309,24%,15,10,8,+10%,0,—,20%
2939,Accomack ›,"2,830","8,757",41,127,—,13,5,7,–34%,0.6,0.23,—
2131,Adair ›,"3,181","14,333",41,185,18%,6,7,7,–19%,0.1,0.18,27%
822,Adair ›,865,"12,095",32,447,34%,1,9,14,+99%,0.1,1.34,39%


In [27]:
all_df.to_csv('data/nyimes_covid_data_by_county.csv')